In [1]:
from google.colab import drive, userdata
import os
drive.mount('/content/drive')
drive_dir = '/content/drive/My Drive/'
data_dir = os.path.join(drive_dir, 'Detection-datasets')
cache_dir = os.path.join(drive_dir, 'model_cache')
openai_api_key = userdata.get('OPENAI_API_KEY')

Mounted at /content/drive


In [2]:
!pip install -q datasets
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# TruthfulQA

In [3]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset('truthful_qa', 'generation')
df = ds['validation'].to_pandas()
TruthfulQA_df = []
for i in range(len(df)):
    row = df.iloc[i]
    for answer in row['correct_answers']:
        TruthfulQA_df.append(pd.DataFrame([{"Statement": f"{row['question']} {answer}.", "Label": 1, 'Type': row['type'], 'Category': row['category'], 'Group': i}]))
    for answer in row['incorrect_answers']:
        TruthfulQA_df.append(pd.DataFrame([{"Statement": f"{row['question']} {answer}.", "Label": 0, 'Type': row['type'], 'Category': row['category'], 'Group': i}]))

TruthfulQA_df = pd.concat(TruthfulQA_df, axis=0).reindex()
TruthfulQA_df = TruthfulQA_df.rename({'Statement': 'Statements', 'Label': 'isTrue'}, axis=1)
TruthfulQA_df.to_csv(os.path.join(data_dir, 'TruthfulQA_prepared.csv'))

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

# Politicians

In [4]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

df = pd.read_csv(os.path.join(data_dir, 'politicians_edge_substitutions.csv'))
ord = OrdinalEncoder(categories=[df['name'].unique().tolist()])
groups = ord.fit_transform(np.array(df['name']).reshape(-1,1))
df['Group'] = groups.astype(np.uint32)
#df = df.sort_values(by='Group', axis=0)

n_groups = 1000

selected_rows = []
for i in df['Group'].unique()[:n_groups]:
  sel = df[df['Group']==i]
  n_true = sel['isTrue'].sum()
  n_false = (1-sel['isTrue']).sum()
  n_sel = min(n_true, n_false)
  selected_rows.append(sel[sel['isTrue']==0][:n_sel])
  selected_rows.append(sel[sel['isTrue']==1][:n_sel])
politicians_df = pd.concat(selected_rows)
politicians_df = politicians_df.reset_index()
politicians_df.to_csv(os.path.join(data_dir, 'Politicians_experiment2_prepared.csv'))
politicians_df

,index,Unnamed: 0,name,nameLabel,education,educationLabel,countryLabel,isTrue,Questions,Statements,Group
0,3,3,Q76,Barack Obama,Q5130976,Cleary University,United States of America,0,Did Barack Obama attend Cleary University?,Barack Obama attended Cleary University.,0
1,4,4,Q76,Barack Obama,Q2001088,Northern State University,United States of America,0,Did Barack Obama attend Northern State Univers...,Barack Obama attended Northern State University.,0
2,5,5,Q76,Barack Obama,Q7895394,University of Florida Athletic Association,United States of America,0,Did Barack Obama attend University of Florida ...,Barack Obama attended University of Florida At...,0
3,0,0,Q76,Barack Obama,Q13371,Harvard University,United States of America,1,Did Barack Obama attend Harvard University?,Barack Obama attended Harvard University.,0
4,1,1,Q76,Barack Obama,Q1346110,Occidental College,United States of America,1,Did Barack Obama attend Occidental College?,Barack Obama attended Occidental College.,0
...,...,...,...,...,...,...,...,...,...,...,...
2625,6072,6267,Q204723,Mirko Cvetković,Q4864983,University of Belgrade Faculty of Economics,Serbia,1,Did Mirko Cvetković attend University of Belgr...,Mirko Cvetković attended University of Belgrad...,997
2626,6079,6274,Q204847,Eugenijus Gentvilas,Q1784353,Šiauliai University,Lithuania,0,Did Eugenijus Gentvilas attend Šiauliai Univer...,Eugenijus Gentvilas attended Šiauliai University.,998
2627,6078,6273,Q204847,Eugenijus Gentvilas,Q658192,Vilnius University,Lithuania,1,Did Eugenijus Gentvilas attend Vilnius Univers...,Eugenijus Gentvilas attended Vilnius University.,998
2628,6085,6280,Q204939,Masaaki Kanda,Q835738,Future University Hakodate,Japan,0,Did Masaaki Kanda attend Future University Hak...,Masaaki Kanda attended Future University Hakod...,999


# Tegmark's datasets
Here I only use "neg_cities", "common_claim_true_false", and "cities". Put these in the `Tegmark` folder.

In [5]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(os.path.join(data_dir, 'Tegmark', 'cities.csv'))
ds_new_name = 'cities_prepared.csv'
df = df.rename({'statement': 'Statements', 'label': 'isTrue'}, axis=1)
df['Group'] = df['city']
df.to_csv(os.path.join(data_dir, ds_new_name))

df = pd.read_csv(os.path.join(data_dir, 'Tegmark', 'neg_cities.csv'))
ds_new_name = 'neg_cities_prepared.csv'
df = df.rename({'statement': 'Statements', 'label': 'isTrue'}, axis=1)
df['Group'] = df['city']
df.to_csv(os.path.join(data_dir, ds_new_name))

df = pd.read_csv(os.path.join(data_dir, 'Tegmark', 'common_claim_true_false.csv'))
ds_new_name = 'common_claim_true_false_prepared.csv'
df = df.rename({'statement': 'Statements', 'label': 'isTrue'}, axis=1)
df['Group'] = np.arange(len(df))
df.to_csv(os.path.join(data_dir, ds_new_name))

# Compute activations

In [6]:
from typing import List
import transformers
from tqdm.notebook import tqdm
import numpy as np
import torch

dtype = np.float16


class Hook:
  # Inspired by https://github.com/saprmarks/geometry-of-truth/blob/main/generate_acts.py
  def __init__(self):
    self.activations = []
    self.idx = -1   # The index of the token we look at the internal state for

  def __call__(self, module, input, **kwargs):
    assert len(input) == 1 and isinstance(input[0], torch.Tensor)
    o = input[0][...,self.idx,:].detach().cpu().numpy().astype(dtype)
    self.activations.append(o)


def clear_hooks(model):
  for layer in model.model.layers:
    layer.self_attn.o_proj._forward_pre_hooks.clear()
    assert not len(layer.self_attn.o_proj._forward_pre_hooks)


def add_hooks(model):
  hooks = []
  handles = []
  for i, layer in enumerate(model.model.layers):
    hook = Hook()
    handle = layer.self_attn.o_proj.register_forward_pre_hook(hook)
    hooks.append(hook)
    handles.append(handle)
  return hooks, handles


def compute_activations(statements: List[str], model: torch.nn.Module, tokenizer) -> np.ndarray:
  """
    Returns:
    - Activations of shape [num_samples, num_layers, num_heads, head_dim]
  """
  # Empty forward hooks just in case something happened.
  clear_hooks(model)
  hooks, handles = add_hooks(model)

  for statement in tqdm(statements, leave=False, desc="Computing activations for each statement"):
    tokens = tokenizer.encode(statement, return_tensors='pt').cuda()    # Wait... How will this end up looking? <inst> Question answer </inst>? Should be <inst> Question </inst> answer
    _ = model(tokens)

  for handle in handles:
    handle.remove()

  activations = []
  for hook in hooks:
    activations.append(np.vstack(hook.activations))
  activations = np.stack(activations, axis=0)                                   # [num_layers, num_samples, n_hidden_dim]

  num_layers = len(model.model.layers)
  num_samples = len(statements)
  num_heads = model.model.layers[0].self_attn.num_heads
  head_dim = model.model.layers[0].self_attn.head_dim
  out = activations.reshape(num_layers, num_samples, num_heads, head_dim)       # [num_layers, num_samples, num_heads, head_dim]
  out = out.transpose(1,0,2,3)                                                  # [num_samples, num_layers, num_heads, head_dim]
  assert (out[2,4,3,:] == activations[4,2,(3*head_dim):(4*head_dim)]).all()
  return out                                                                    # [num_layers, num_samples, num_heads, head_dim]

In [7]:
import torch, gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from typing import List, Tuple
import pandas as pd
from tqdm.notebook import tqdm

def compute_activations_on_datasets(model_name: str, datasets: List[Tuple[str, pd.DataFrame]], model_setup_fn=None):
  """
    Should traverse datasets, not models (so we don't have to re-load the same model multiple times - this is expensive)
  """

  # Load 4 bit quantized model
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.float16,
  )
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", cache_dir=cache_dir, quantization_config=quantization_config, trust_remote_code=True)
  tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
  if model_setup_fn is not None:
    model, tokenizer = model_setup_fn(model, tokenizer)

  # Compute activations and save
  for dataset_name, df in tqdm(datasets, leave=False, desc="Datasets"):
    file_name_str = f"Activations_{dataset_name}_{model_name.split('/')[-1]}.npz"
    activations = compute_activations(df["Statements"], model, tokenizer)
    np.savez(os.path.join(data_dir, file_name_str), activations=activations, labels=df['isTrue'], group=(df['Group'] if 'Group' in df.columns else None))

    # Check if correctly saved
    a_ = np.load(os.path.join(data_dir, file_name_str))['activations']
    assert np.all(activations == a_)

  # Clean up model
  model.cpu()
  del model
  torch.cuda.empty_cache()
  gc.collect()


#compute_activations_on_dataset("meta-llama/Meta-Llama-3-8B", "Politicians", df_concat_politicians)

In [8]:
# Note: These chat templates are not used right now... Just added them here because they're needed for fine-tuning, hence maybe also for this task later on...
def opt_setup_fn(model, tokenizer):
  # Patch the attention heads to make it compatible with the rest (this one is needed!)
  model.model.layers = model.model.decoder.layers
  for l in model.model.layers:
    l.self_attn.o_proj = l.self_attn.out_proj
  return model, tokenizer

In [9]:
datasets = [(ds_file_name.replace('_prepared.csv', ''), pd.read_csv(os.path.join(data_dir, ds_file_name))) for ds_file_name in ["TruthfulQA_prepared.csv", "Politicians_experiment2_prepared.csv", "neg_cities_prepared.csv", "common_claim_true_false_prepared.csv", "cities_prepared.csv"]] #filter(lambda x: '.csv' in x, os.listdir(data_dir))]
datasets = list(map(lambda ds: (ds[0], (ds[1][0:4000] if len(ds[1])>4000 else ds[1])), datasets))
models = ["facebook/opt-125m", "facebook/opt-350m", "facebook/opt-2.7b", "meta-llama/Llama-2-7b-chat-hf", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B"]
model_setup_fns = [opt_setup_fn, opt_setup_fn, opt_setup_fn, None, None, None]

for model, model_setup_fn in tqdm(list(zip(models, model_setup_fns)), leave=False, desc="Models"):
  # Remember model setup function!
  ## For opt: Set model.model.layers = model.model.decoder.layers
  compute_activations_on_datasets(model, datasets, model_setup_fn=model_setup_fn)

Models:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/2630 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/1496 [00:00<?, ?it/s]

...

Computing activations for each statement:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/2630 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/1496 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/1496 [00:00<?, ?it/s]

In [10]:
[i for i, _ in datasets]

['TruthfulQA',
 'Politicians_experiment2',
 'neg_cities',
 'common_claim_true_false',
 'cities']